# Group Chat Orchestration - Round Robin Group Chat Manager

Group chat orchestration models a collaborative conversation among agents, optionally including a human participant. A group chat manager coordinates the flow, determining which agent should respond next and when to request human input. This pattern is powerful for simulating meetings, debates, or collaborative problem-solving sessions.

* [Agent-orchestration Guide - Group Chat Orchestration](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-orchestration/group-chat?pivots=programming-language-python)
* [Original Code](https://github.com/microsoft/semantic-kernel/blob/main/python/samples/getting_started_with_agents/multi_agent_orchestration/step3_group_chat.py)

## Sample Description 

The following sample demonstrates how to create a group chat orchestration with a default
round robin manager for controlling the flow of conversation in a round robin fashion.

Think of the group chat manager as a state machine, with the following possible states:
- Request for user message
- Termination, after which the manager will try to filter a result from the conversation
- Continuation, at which the manager will select the next agent to speak

This sample demonstrates the basic steps of creating and starting a runtime, creating
a group chat orchestration with a group chat manager, invoking the orchestration,
and finally waiting for the results.

There are two agents in this orchestration: a writer and a reviewer. They work iteratively
to refine a slogan for a new electric SUV.

In [22]:
# Libraries
from dotenv import load_dotenv
import os
import asyncio

from semantic_kernel.agents import Agent, ChatCompletionAgent, GroupChatOrchestration, RoundRobinGroupChatManager
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatMessageContent

In [23]:
#load variables
load_dotenv()

# Variables - Azure Services
azopenai_ep=os.environ["AZURE_OPENAI_ACCOUNT"]
azopenai_key=os.environ["AZURE_OPENAI_KEY"]
azopenai_model=os.environ["AZURE_OPENAI_MODEL"]

### Create an instance of AzureChatCompletion with the provided configuration

In [24]:
az_chat_completion = AzureChatCompletion(
    deployment_name=azopenai_model,
    api_key=azopenai_key,
    endpoint=azopenai_ep 
)

### Create the agents

In [25]:
def get_agents() -> list[Agent]:
    """Return a list of agents that will participate in the group style discussion.

    Feel free to add or remove agents.
    """
    writer = ChatCompletionAgent(
        name="Writer",
        description="A content writer.",
        instructions=(
            "You are an excellent content writer. You create new content and edit contents based on the feedback."
        ),
        service=az_chat_completion
    )
    reviewer = ChatCompletionAgent(
        name="Reviewer",
        description="A content reviewer.",
        instructions=(
            "You are an excellent content reviewer. You review the content and provide feedback to the writer."
        ),
        service=az_chat_completion
    )

    # The order of the agents in the list will be the order in which they will be picked by the round robin manager
    return [writer, reviewer]

### Optional: Observe Agent Responses

You can define a callback to observe and print the output from each agent as the sequence progresses.

In [26]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents."""
    print(f"**{message.name}**\n{  message.content}\n\n")

### Set Up the Group Chat Orchestration

Create a GroupChatOrchestration object, passing in the agents, a group chat manager (in this case a RoundRobinGroupChatManager), and the response callback. The manager controls the flow—here, it alternates turns in a round-robin fashion for a set number of rounds.

In [27]:
# 1. Create a group chat orchestration with a round robin manager
agents = get_agents()
group_chat_orchestration = GroupChatOrchestration(
    members=agents,
    # max_rounds is odd, so that the writer gets the last round
    manager=RoundRobinGroupChatManager(max_rounds=5),
    agent_response_callback=agent_response_callback,
)

### Start the Runtime

Start the runtime to manage agent execution.

In [28]:
# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

**Writer**
Certainly! Based on the inspirations provided, here’s a company name suggestion:

**PradoTech Real Solutions**

This name combines "Prado" and "Technology" to emphasize innovation, while "Real Solutions" ties in with real estate, offering a sense of practicality, trust, and commitment to quality. It captures the essence of modern innovation applied to the real estate industry, with a subtle nod to Costa Rica's forward-thinking and reliable spirit. 

Would you like any additional iterations or refinements?


**Reviewer**
Thank you for the submission! Here's my review of the proposed company name:

**PradoTech Real Solutions**

**Strengths:**
1. **Relevance:** The name effectively incorporates "Prado" as requested, and connects it to "Technology" to emphasize the innovation aspect. 
2. **Clarity:** It clearly signals to potential customers that the company is involved with technology and solutions, which is fitting for the real estate industry.
3. **Trust Factor:** The phrase 

### Invoke the Orchestration

Invoke the orchestration with your initial task (e.g., "Create a slogan for a new electric SUV..."). The agents will take turns responding, refining the result.

In [29]:
orchestration_result = await group_chat_orchestration.invoke(    
    task="Create a Company Name using as inspirations the following words: 'Prado', 'Technology', 'Innovation', 'Real States', 'Costa Rica', 'Trust'. \
        Make sure the name contains the word 'Prado' and is related to technology and real states.",
    runtime=runtime,
)


### Collect Results and Stop the Runtime

In [30]:
# 4. Wait for the results
value = await orchestration_result.get()
print(f"***** Result *****\n{value}")

# 5. Stop the runtime after the invocation is complete
await runtime.stop_when_idle()

***** Result *****
Thank you for the review! Incorporating your feedback, I’ll refine the names further to balance clarity, innovation, relevance to real estate, and a nod to Costa Rica. Here’s a fresh, polished batch of company name suggestions:

1. **PradoSmart Realty**  
   - A modern, tech-inspired name that remains concise and easy to remember, blending innovation with real estate.

2. **Prado Costa Innovations**  
   - Highlights Costa Rica’s identity while emphasizing the company’s innovative, tech-driven focus.

3. **PradoTech Properties**  
   - A straightforward fusion that combines modern technology with real estate, ensuring clarity of purpose.

4. **PradoTrust Realty**  
   - Puts trust front and center, creating a sense of reliability while firmly connecting with the real estate sphere.

5. **PradoVista Realty**  
   - "Vista" evokes images of scenic beauty and forward-thinking, nodding to Costa Rica while suggesting vision and innovation.

6. **PradoEstate Tech**  
   - 